# Data Cleaning

At the end of this notebook there are dataframes that are ready to be placed into the GAN

### Import Packages

In [1]:
import pandas as pd

import argparse
import json
import logging
import os

import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision
import torchvision.models
import torchvision.transforms as transforms

from typing import List

## Set up logger to get details of errors
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

### Import data

In [2]:
# Importing Data
rawdata = pd.read_csv('s3://sagemaker-studio-dfml0t4nnx4/clean_data.csv')
rawdata.drop(columns = ['lmfdb_label', 'rank'])

coef = rawdata.drop(columns = ['Unnamed: 0','lmfdb_label'])
coef['a1'] = pd.to_numeric(coef['a1'], errors='coerce')
coef['a6'] = pd.to_numeric(coef['a6'], errors='coerce')
coef = coef.dropna()

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
coef.describe()

,a1,a2,a3,a4,a6,rank
count,480517.000000,480517.000000,480517.000000,4.805170e+05,4.805170e+05,480517.000000
mean,0.496051,-0.059665,0.350820,-3.700028e+09,4.902672e+15,0.724176
std,0.499985,0.800867,0.477227,8.032681e+11,5.447199e+18,0.648116
min,0.000000,-1.000000,0.000000,-4.234064e+14,-9.801624e+20,0.000000
25%,0.000000,-1.000000,0.000000,-2.117900e+04,-4.526300e+04,0.000000
50%,0.000000,0.000000,0.000000,-8.140000e+02,6.000000e+00,1.000000
75%,1.000000,1.000000,1.000000,-1.200000e+01,5.264400e+04,1.000000
max,1.000000,1.000000,1.000000,9.961486e+11,3.353391e+21,3.000000


### Making Data Binary

Making the data binary allows us to use the sigmoid function as the output layer of the nn, and it helps us make sure that the output is in whole numbers rather than decimals. 

In [6]:
# Create new dataframe to work with
binaryraw = coef

# Set limits for coefficients
binaryraw = binaryraw[binaryraw['a4'] > -(2**13)]
binaryraw = binaryraw[binaryraw['a4'] < 2**13]
binaryraw = binaryraw[binaryraw['a6'] > -(2**13)]
binaryraw = binaryraw[binaryraw['a6'] < 2**13]
binarycoef = binaryraw

In [7]:
binarycoef.describe()

,a1,a2,a3,a4,a6,rank
count,177460.000000,177460.000000,177460.000000,177460.000000,177460.000000,177460.000000
mean,0.465316,-0.036093,0.355889,-103.498067,28.793244,0.828626
std,0.498797,0.805042,0.478783,465.556261,2573.171074,0.682717
min,0.000000,-1.000000,0.000000,-8141.000000,-8191.000000,0.000000
25%,0.000000,-1.000000,0.000000,-233.000000,-547.000000,0.000000
50%,0.000000,0.000000,0.000000,-52.000000,2.000000,1.000000
75%,1.000000,1.000000,1.000000,11.000000,619.000000,1.000000
max,1.000000,1.000000,1.000000,8191.000000,8191.000000,3.000000


In [8]:
# Binary for a1
binarycoef['a1b'] = binarycoef['a1'].astype('int')

# Binary for a2
# The first coefficient is for the sign, 0 means positive, 1 means negative
binarycoef['a2b1'] = binarycoef['a2'].apply(lambda x : 0 if x != -1 else 1) 
binarycoef['a2b2'] = binarycoef['a2'].apply(lambda x : 1 if x != 0 else 0) 

# Binary for a3
binarycoef['a3b'] = binarycoef['a3'].apply(lambda x : 1 if x != 0 else 0)

# Binary for a4
### First I find whether it is positive or negative
binarycoef['a4b1'] = binarycoef['a4'].apply(lambda x : 1 if x < 0 else 0)
### Then I create a function that produces a binary list representation of an integer 
def create_binary_list_from_int(number: int) -> List[int]:
    """Creates a list of the binary representation of a positive integer

    Args:
        number: An integer

    Returns:
        The binary representation of the provided positive integer number as a list.
    """
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")
    data = [int(x) for x in list(bin(number))[2:]]
    data = ([0] * (13 - len(data))) + data

    return data
### Then I run the integer to binary list function on the value of a4 and create column 'a4b'
binarycoef1 = binarycoef.copy()
binarycoef1['a4b'] = (binarycoef['a4'].abs().astype(int)).apply(create_binary_list_from_int)
### Then create columns from that list
a4binary = pd.DataFrame(binarycoef1['a4b'].to_list(), columns = ['a4b2', 'a4b3', 'a4b4', 'a4b5', 'a4b6', 'a4b7', 'a4b8', 'a4b9', 
                                                                'a4b10', 'a4b11', 'a4b12', 'a4b13', 'a4b14'])
rank_df1 = pd.concat([binarycoef.reset_index(), a4binary], axis = 1)

# Binary for a6 (similar process to a4)
rank_df1['a6b1'] = rank_df1['a6'].apply(lambda x : 1 if x < 0 else 0)
rank_values2 = binarycoef.copy(deep=False)
rank_values2['a6b'] = (binarycoef['a6'].abs().astype(int)).apply(create_binary_list_from_int)
a6binary = pd.DataFrame(rank_values2['a6b'].to_list(), columns = ['a6b2', 'a6b3', 'a6b4', 'a6b5', 'a6b6', 'a6b7', 'a6b8', 'a6b9', 
                                                                 'a6b10', 'a6b11', 'a6b12', 'a6b13', 'a6b14'])
coef_df_binary1 = pd.concat([rank_df1.reset_index(), a6binary], axis = 1)

### Create DataFrame with only variables of interest

In [9]:
coef_df_binary = coef_df_binary1[['a1b', 'a2b1', 'a2b2', 'a3b', 'a4b1', 'a4b2', 'a4b3', 'a4b4', 'a4b5', 'a4b6', 'a4b7', 'a4b8', 'a4b9', 
                                  'a4b10', 'a4b11', 'a4b12', 'a4b13', 'a4b14', 'a6b1', 'a6b2', 'a6b3', 'a6b4', 'a6b5', 'a6b6', 'a6b7', 
                                  'a6b8', 'a6b9', 'a6b10', 'a6b11', 'a6b12', 'a6b13', 'a6b14', 'rank']]

### Filtering data 

Making all the curves rank 1

In [10]:
coef_df_binary = coef_df_binary[coef_df_binary['rank'] == 1]
coef_df_binary = coef_df_binary.drop(columns = ['rank'])

In [11]:
coef_df_binary.describe()

,a1b,a2b1,a2b2,a3b,a4b1,a4b2,a4b3,a4b4,a4b5,a4b6,...,a6b5,a6b6,a6b7,a6b8,a6b9,a6b10,a6b11,a6b12,a6b13,a6b14
count,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,...,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000,91430.000000
mean,0.471465,0.340829,0.648934,0.356294,0.685081,0.001870,0.006989,0.021295,0.112108,0.197977,...,0.303675,0.353943,0.394181,0.431937,0.457333,0.482861,0.464705,0.484010,0.419676,0.468479
std,0.499188,0.473991,0.477306,0.478906,0.464486,0.043207,0.083308,0.144367,0.315501,0.398477,...,0.459846,0.478194,0.488677,0.495348,0.498179,0.499709,0.498755,0.499747,0.493509,0.499008
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Download DataFrames

In [13]:
coef_df_binary.to_csv('rank_1_curves.csv')